In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import dataloader as data

In [5]:
person_index = 30
viewpoint_index = 1
gender = "*"
data.load_nosepoint(person_index, viewpoint_index, gender)

array([0.5542087 , 0.62599987], dtype=float32)

In [ ]:
training_set = []
gender = "*" # select both gen
for person_index in range(1,32+1):
    for viewpoint_index in range(1,6+1):
        